In [7]:
import pandas as pd
import faiss
import numpy as np
import ast
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import torch
import gc
from ast import literal_eval
from transformers import AutoTokenizer
from peft import AutoPeftModelForCausalLM
from torch.utils.data import Dataset

ds = load_dataset("lcw99/wikipedia-korean-20240501")
test_df = pd.read_csv('/data/ephemeral/pro-nlp-generationfornlp-nlp-13/data/test.csv')

In [8]:
wiki_df = pd.DataFrame(ds['train']);wiki_df

,title,section_titles,section_texts,text
0,지미 카터,"[Introduction, 약력, 생애, 대통령 재임, 퇴임 이후, 평가, 같이 보...","[\n\n'''제임스 얼 “지미” 카터 주니어'''(, 1924년 10월 1일~)는...","'''제임스 얼 “지미” 카터 주니어'''(, 1924년 10월 1일~)는 민주당 ..."
1,수학,"[Introduction, 역사, 세부 분야, 영향, 같이 보기, 참고 문헌, 외부...","[\n\n'''수학'''(, , '''math''')은 수, 양, 구조, 공간, 변...","'''수학'''(, , '''math''')은 수, 양, 구조, 공간, 변화 등의 ..."
2,수학 상수,"[Introduction, 수학 상수표, 관련 상수들, 기타 상수들, 같이 보기]","[\n'''수학'''에서 '''상수'''란 그 값이 변하지 않는 불변량으로, 변수의...","'''수학'''에서 '''상수'''란 그 값이 변하지 않는 불변량으로, 변수의 반대..."
3,문학,"[Introduction, 일반적인 문학의 분류, 대중문학의 분류, 문학 사조, 문...","[\n\n\n파일:Fragonard, The Reader.jpg|섬네일|250px|...","파일:Fragonard, The Reader.jpg|섬네일|250px|장오노레 프라..."
4,나라 목록,"[Introduction, 기준, 남극, EU, 참고, 몰타 기사단, [[마이크로네...",[\n스위스 제네바에 있는 국제 연합 회원국 및 비회원 GA 옵서버의 국기\n\n이...,스위스 제네바에 있는 국제 연합 회원국 및 비회원 GA 옵서버의 국기\n이 목록에 ...
...,...,...,...,...
515420,송곳부리도요,"[Introduction, 생김새, 서식지]","[\n\n'''송곳부리도요'''(, )는 도요목 도요과에 속하는 나그네새이다.\n,...","'''송곳부리도요'''(, )는 도요목 도요과에 속하는 나그네새이다.\n흰 눈썹선과..."
515421,흑양,"[Introduction, 설명, 분포, 외부 링크, 참고문헌]","[\n\n\n\n'''흑양'''(黑楊. ''Populus nigra'')은 유럽, ...","'''흑양'''(黑楊. ''Populus nigra'')은 유럽, 서남아시아 및 중..."
515422,니시하타 유키오,"[Introduction, 저서, 외부 링크]","[\n'''니시하타 유키오'''(, 1946년~)는 일본의 언어학자이다.\n, * ...","'''니시하타 유키오'''(, 1946년~)는 일본의 언어학자이다.\n* 「고대 조..."
515423,안드리 스타드니크,"[Introduction, 개인사, 외부 링크]","[\n\n'''안드리 볼로디미로비치 스타드니크'''(, 1982년 4월 15일~)는...","'''안드리 볼로디미로비치 스타드니크'''(, 1982년 4월 15일~)는 우크라이..."


In [9]:
records = []
for _, row in test_df.iterrows():
    problems = literal_eval(row['problems'])
    record = {
        'id': row['id'],
        'paragraph': row['paragraph'],
        'question': problems['question'],
        'choices': problems['choices'],
        'answer': problems.get('answer', None),
        'question_plus': problems.get('question_plus', None),
    }
    records.append(record)

test_df = pd.DataFrame(records)
test_df['num_choices'] = test_df['choices'].apply(len)

In [16]:
test_df = test_df[test_df.num_choices==4]

In [17]:
test_df

,id,paragraph,question,choices,answer,question_plus,num_choices
425,generation-for-nlp-997,자유인이 항상 가장 애써서 지켜온 권리 중 하나는 자신의 근면에 대한 보상을 즐길 ...,위의 구절에서 쿨리지는 다음 중 무엇에 반응합니까?,"[전임자 워렌 G. 하딩의 경제정책, 러시아 혁명 이후 미국 내 성장한 급진 정치,...",,None,4
426,generation-for-nlp-990,"1980년대는 혼란, 갈등, 변화 속에서 탄생했습니다. 지금은 우리의 관심과 가치가...",카터가 설명한 상황으로 인해 가장 직접적으로 이어진 것은 다음 중 어떤 상황입니까?,"[북대서양조약기구(NATO) 창설, 차기 대선에서 카터의 패배, 미국의 중동 침공,...",,None,4
428,generation-for-nlp-1032,"""오명으로 남을 어제 1941년 12월 7일 미국은 일본 제국의 해공군으로부터 갑작...",위 진술이 작성되었을 가능성이 가장 높은 시점은 다음 중 어떤 사건 이후입니까?,"[진주만 폭격, 미국 메인호 침몰, U.S.S. 리버티호 폭격, 복서 반란]",,None,4
430,generation-for-nlp-1343,"너는 미합중국이다, 인디언의 혈통이며 예수 그리스도에게 기도하고 스페인어를 말하는 ...","시인이 ""고대부터 시인들이 있었고, . . . 별을 바라보던 우리 미국"" 을 논한 ...","[라틴아메리카는 시에 뛰어났다., 라틴 아메리카는 상당한 과학적 기여를 이루었다.,...",,None,4
433,generation-for-nlp-1292,"그는 모든 작업과 욕망, 모든 향기, 모든 맛을 담고 있다. 그는 모든 우주를 품으...",위 인용문에 기반할 때 다음 중 화자의 종교에 대해 올바른 것은?,"[구원은 의식을 올바르게 끝내는 데에 기반한다., 내세에 대한 기대가 존재한다., ...",,None,4
...,...,...,...,...,...,...,...
857,generation-for-nlp-1252,"자아를 아는 것은 전차에 앉아 있는 것이고, 육체는 전차이며, 지성은 전차의 운전수...",다음 중 발췌문에서 전달된 메시지와 불교의 환생에 관한 교리가 힌두교와 가장 닮은 ...,"[업보에 대한 개념을 가지고 있다., 사후 천국에 대한 개념을 제안한다., 모든 신...",,None,4
858,generation-for-nlp-609,"""하지만 친애하는 팡글로스."" 캉디드가 말했다. ""내가 어찌 당신을 다시 볼 수 있...",팡글로스를 교수형에 처하지 못한 이단심판 집행자를 조롱한 것은 시대의 어떤 모습을 ...,"[토르케마다의 개인적 경험, 이슬람 생활 양식 수용, 가톨릭 교리에 대한 일반적 거...",,None,4
859,generation-for-nlp-906,이웃집 개가 갑자기 짖어대자 아이가 겁을 먹습니다. 엄마가 아이를 안으면 안정을 되...,이는 다음 중 어떤 생물학적 프로세스가 발생하기 때문입니까?,"[부교감신경계는 제어를 재개하고 교감 반응을 원래의 상태로 복귀시킵니다., 교감신경...",,None,4
860,generation-for-nlp-731,소비자 물가지수(CPI)가 2% 상승하고 명목 소득이 8% 증가하면 실질 소득은 대략,소비자 물가지수(CPI)가 2% 상승하고 명목 소득이 8% 증가하면 실질 소득은 대략?,"[4% 증가한다., 4% 감소한다., 6% 증가한다., 6% 감소한다.]",,None,4


In [11]:
class ChunkDataset(Dataset):
    """Chunk 방식"""
    def __init__(self, dataset, tokenizer, max_length, overlap=50):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.data = []
        
        print("청크 생성 중...")
        for item in dataset:
            question = item['question']
            context = item['context']
            
            c_tokens = tokenizer.encode(context, add_special_tokens=False)
            
            if len(c_tokens) > max_length - 2:
                stride = max_length - overlap - 2
                for i in range(0, len(c_tokens), stride):
                    chunk = c_tokens[i:i + max_length - 2]
                    if len(chunk) >= 10:
                        chunk_text = tokenizer.decode(chunk)
                        self.data.append({
                            'question': question,
                            'context': chunk_text,
                            'original_context': context
                        })
            else:
                self.data.append({
                    'question': question,
                    'context': context,
                    'original_context': context
                })
        
        print(f"청크 생성 완료: {len(self.data)}개")
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        
        q_encoding = self.tokenizer(
            item['question'],
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt'
        )
        
        c_encoding = self.tokenizer(
            item['context'],
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt'
        )
        
        return {
            'q_input_ids': q_encoding['input_ids'].squeeze(),
            'q_attention_mask': q_encoding['attention_mask'].squeeze(),
            'c_input_ids': c_encoding['input_ids'].squeeze(),
            'c_attention_mask': c_encoding['attention_mask'].squeeze(),
            # 'context': item['original_context']
        }

In [13]:
MODEL_NAME = "dragonkue/BGE-m3-ko"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
def chunk_tokenize(text, tokenizer, max_length=512, stride=128):
    """텍스트를 토큰화한 후 청킹"""
    tokens = tokenizer.encode(text, add_special_tokens=False)
    
    chunks = []
    for i in range(0, len(tokens), max_length - stride):
        chunk = tokens[i:i + max_length]
        if len(chunk) >= 10:  # 너무 짧은 청크는 제외
            chunks.append(chunk)
    
    return chunks

In [ ]:
# wiki_df의 text 청킹
print("wiki_df text 청킹 중...")
wiki_chunks = []
for idx, row in tqdm(wiki_df.iterrows(), total=len(wiki_df)):
    chunks = chunk_tokenize(row['text'], tokenizer, max_length=512, stride=128)
    for chunk_idx, chunk in enumerate(chunks):
        wiki_chunks.append({
            'original_idx': idx,
            'title': row['title'],
            'chunk_idx': chunk_idx,
            'token_ids': chunk,
            'num_tokens': len(chunk)
        })

wiki_chunks_df = pd.DataFrame(wiki_chunks)
print(f"wiki_df 청킹 완료: {len(wiki_chunks_df)}개 청크 생성")

In [ ]:
# 임베딩 모델 로드
model = SentenceTransformer(MODEL_NAME)
model.eval()
if torch.cuda.is_available():
    model = model.cuda()
    
print(f"Model loaded on {'cuda' if torch.cuda.is_available() else 'cpu'}")

In [ ]:
# wiki 청크 임베딩 생성
print("wiki 청크 임베딩 생성 중...")
wiki_texts = [tokenizer.decode(tokens) for tokens in wiki_chunks_df['token_ids'].tolist()]

batch_size = 32
wiki_embeddings = []

for i in tqdm(range(0, len(wiki_texts), batch_size)):
    batch = wiki_texts[i:i+batch_size]
    with torch.no_grad():
        embeddings = model.encode(batch, convert_to_numpy=True, show_progress_bar=False)
    wiki_embeddings.append(embeddings)
    
    # 메모리 관리
    if i % 1000 == 0:
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

wiki_embeddings = np.vstack(wiki_embeddings)
print(f"wiki 임베딩 완료: {wiki_embeddings.shape}")

In [ ]:
# FAISS 인덱스 생성
print("FAISS 인덱스 생성 중...")
dimension = wiki_embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)  # Inner Product (cosine similarity)

# 정규화 (cosine similarity를 위해)
faiss.normalize_L2(wiki_embeddings)
index.add(wiki_embeddings)

print(f"FAISS 인덱스 생성 완료: {index.ntotal}개 벡터")

In [ ]:
# test_df paragraph 토큰화 (512 토큰, 패딩 허용)
print("test_df paragraph 토큰화 중...")
test_tokenized = []
for paragraph in tqdm(test_df['paragraph'].tolist()):
    tokens = tokenizer.encode(
        paragraph,
        add_special_tokens=False,
        max_length=512,
        truncation=True,
        padding='max_length'
    )
    test_tokenized.append(tokens)

test_df['token_ids'] = test_tokenized
test_df['num_tokens'] = test_df['token_ids'].apply(len)
print(f"test_df 토큰화 완료: {len(test_df)}개")

# test_df paragraph 임베딩 생성
print("test_df paragraph 임베딩 생성 중...")
test_texts = [tokenizer.decode(tokens) for tokens in test_tokenized]

test_embeddings = []
for i in tqdm(range(0, len(test_texts), batch_size)):
    batch = test_texts[i:i+batch_size]
    with torch.no_grad():
        embeddings = model.encode(batch, convert_to_numpy=True, show_progress_bar=False)
    test_embeddings.append(embeddings)

test_embeddings = np.vstack(test_embeddings)
faiss.normalize_L2(test_embeddings)

print(f"test 임베딩 완료: {test_embeddings.shape}")

In [ ]:
# FAISS로 유사한 wiki 청크 검색 (Top 5)
print("유사한 wiki 청크 검색 중...")
k = 5  # 상위 5개
distances, indices = index.search(test_embeddings, k)

print(f"검색 완료: 각 test_df에 대해 {k}개의 유사 청크 발견")

In [ ]:
# 결과 정리
results = []
for i, (test_idx, row) in enumerate(test_df.iterrows()):
    top_k_indices = indices[i]
    top_k_distances = distances[i]
    
    retrieved_chunks = []
    retrieved_texts = []
    for rank, (chunk_idx, score) in enumerate(zip(top_k_indices, top_k_distances)):
        chunk_info = wiki_chunks_df.iloc[chunk_idx]
        text = tokenizer.decode(chunk_info['token_ids'])
        
        retrieved_chunks.append({
            'rank': rank + 1,
            'score': float(score),
            'title': chunk_info['title'],
            'chunk_idx': chunk_info['chunk_idx'],
            'token_ids': chunk_info['token_ids'],
            'text': text
        })
        retrieved_texts.append(text)
    
    results.append({
        'test_idx': test_idx,
        'id': row['id'],
        'paragraph': row['paragraph'],
        'question': row['question'],
        'retrieved_chunks': retrieved_chunks,
        'retrieved_texts': retrieved_texts
    })

results_df = pd.DataFrame(results)
print(f"결과 정리 완료: {len(results_df)}개 항목")
results_df.head()

In [ ]:
# 현재 메모리에 있는 results_df에서 retrieved_chunks의 text만 추출
# retrieved_chunks는 이미 리스트[딕셔너리] 형태

retrieved_texts_list = []
for chunks in results_df['retrieved_chunks']:
    if isinstance(chunks, str):
        # CSV에서 읽은 경우 문자열이므로 파싱
        chunks = literal_eval(chunks)
    texts = [chunk['text'] for chunk in chunks]
    retrieved_texts_list.append(texts)

results_df['retrieved_texts'] = retrieved_texts_list
results_df[['id', 'question', 'retrieved_texts']].head()

In [ ]:
# retrieved_chunks에서 유사도(score), text, paragraph 쌍으로 df 생성
vector_db_data = []

for idx, row in results_df.iterrows():
    test_id = row['id']
    question = row['question']
    paragraph = row['paragraph']
    retrieved_chunks = row['retrieved_chunks']
    
    # retrieved_chunks가 문자열인 경우 파싱
    if isinstance(retrieved_chunks, str):
        retrieved_chunks = literal_eval(retrieved_chunks)
    
    for chunk in retrieved_chunks:
        vector_db_data.append({
            'test_id': test_id,
            'question': question,
            'paragraph': paragraph,
            'rank': chunk['rank'],
            'score': chunk['score'],
            'title': chunk['title'],
            'retrieved_text': chunk['text']
        })

vector_db_df = pd.DataFrame(vector_db_data)
print(f"벡터 DB용 데이터 생성 완료: {len(vector_db_df)}개 텍스트")
vector_db_df.head(10)

In [ ]:
# 현재 데이터의 토큰 수 확인

# 1. Wiki chunks 토큰 수
wiki_total_tokens = wiki_chunks_df['num_tokens'].sum()
wiki_num_chunks = len(wiki_chunks_df)

# 2. Test data 토큰 수
test_total_tokens = test_df['num_tokens'].sum()
test_num_samples = len(test_df)

# 3. Retrieved texts 토큰 수 (vector_db_df 기준)
retrieved_texts = vector_db_df['retrieved_text'].tolist()
retrieved_tokens = [len(tokenizer.encode(text, add_special_tokens=False)) for text in retrieved_texts]
retrieved_total_tokens = sum(retrieved_tokens)
retrieved_num_texts = len(retrieved_texts)

print("=" * 60)
print("📊 토큰 수 통계")
print("=" * 60)
print(f"\n1. Wiki Chunks:")
print(f"   - 청크 개수: {wiki_num_chunks:,}개")
print(f"   - 총 토큰 수: {wiki_total_tokens:,} tokens")
print(f"   - 평균 토큰/청크: {wiki_total_tokens/wiki_num_chunks:.1f} tokens")

print(f"\n2. Test Data (Paragraphs):")
print(f"   - 샘플 개수: {test_num_samples:,}개")
print(f"   - 총 토큰 수: {test_total_tokens:,} tokens")
print(f"   - 평균 토큰/샘플: {test_total_tokens/test_num_samples:.1f} tokens")

print(f"\n3. Retrieved Texts (검색된 위키 텍스트):")
print(f"   - 텍스트 개수: {retrieved_num_texts:,}개")
print(f"   - 총 토큰 수: {retrieved_total_tokens:,} tokens")
print(f"   - 평균 토큰/텍스트: {retrieved_total_tokens/retrieved_num_texts:.1f} tokens")

print(f"\n4. 전체 임베딩해야 할 토큰:")
print(f"   - Wiki: {wiki_total_tokens:,} tokens")
print(f"   - Test: {test_total_tokens:,} tokens")
print(f"   - 합계: {wiki_total_tokens + test_total_tokens:,} tokens")

# OpenAI API 비용 예상 (text-embedding-3-small 기준: $0.02 / 1M tokens)
total_for_embedding = wiki_total_tokens + test_total_tokens
openai_cost = (total_for_embedding / 1_000_000) * 0.02

print(f"\n💰 예상 비용 (OpenAI text-embedding-3-small):")
print(f"   - 총 임베딩 토큰: {total_for_embedding:,} tokens")
print(f"   - 예상 비용: ${openai_cost:.4f}")
print("=" * 60)